In [2]:
import streamlit

In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.7,
    # max_tokens=1000
)

res = llm.invoke("hello")
print(res)

content='Hello. How can I help you today?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.028439477, 'prompt_time': 0.001720475, 'queue_time': 0.067997378, 'total_time': 0.030159952}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9a8b91ba77', 'finish_reason': 'stop', 'logprobs': None} id='run--ad1faee5-ec1a-4c96-96c8-5506d6e193e8-0' usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46}


In [2]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from typing import Literal, TypedDict, List, Optional, Dict
from typing_extensions import Annotated
from langgraph.graph import START, StateGraph
from langchain_core.vectorstores import InMemoryVectorStore
import json
import logging
import time
from dotenv import load_dotenv
import getpass
import os
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_tavily import TavilySearch
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
import logging

# from prompts import (Questionnaire_prompt, report_planner_query_writer)

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    level=logging.INFO,
    handlers=[logging.FileHandler("TS.log"), logging.StreamHandler()]
)

logger = logging.getLogger(__name__)

START hello

Hello! My name is Devin, a Talent Scout Assistant here at SkillZilla. It's great to speak with you today. Thanks for taking the time to apply for the [Job Title] position. I'm assisting Aria, our HR Manager, with the candidate screening process. 

Could you please tell me a little about yourself, your background, and what interests you about this particular role at SkillZilla?  




In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Based "
)

chain = prompt_template | llm

res = chain.invoke({"input": "hello"})
print(res.content)

## Resume parser 

In [3]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import List, TypedDict
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

import os
os.environ["USER_AGENT"] = "TalentScout" 

def resume_loader(path_to_resume: str):

    loader = DirectoryLoader(
                path_to_resume,
                glob="**/*.pdf",
                loader_cls=PyMuPDFLoader,
                show_progress=True,  # Optional: Displays a progress bar
                use_multithreading=True  # Optional: Enables multithreading for faster loading
            )

    langchian_loader = loader.load() 
    resume_data =langchian_loader[0].page_content
    return resume_data



resume_data = resume_loader("user_data")
print(resume_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.
100%|██████████| 1/1 [00:00<00:00,  4.30it/s]

Shaswat Singh 
AI DEVELOPER 
Lucknow, UP, India | 9140529926 | Shaswatsingh910@gmail.com | LinkedIn | GitHub| Medium | Website 
 
Objective 
An aspiring AI developer and software engineer with a passion for building intelligent systems 
that automate and enhance real-world processes. Experienced in developing AI-powered 
solutions, fine-tuning models, and deploying applications on cloud infrastructure. Skilled in 
integrating natural language processing (NLP), computer vision, and large language models into 
innovative projects. 
 
Education 
CGPA: 6.9 
 
University of Lucknow | Bachelor of Computer Application                       2022-2025 
 
Relevant Coursework 
Data Structures, Software Engineering, Algorithms, Database Management, Computer Networks, 
Operating System, Artificial Intelligence, Machine Learning 
Skills & abilities 
 
• 
Programming Languages: Python 
• 
AI/ML Frameworks: Hugging Face, Langhian, Llama Index, Unsloth, Transformers, FastAPI, 
Numpy, pandas, seaborn, P

In [2]:
print(type(resume_data))

<class 'str'>


In [4]:
print(resume_data)

Shaswat Singh 
AI DEVELOPER 
Lucknow, UP, India | 9140529926 | Shaswatsingh910@gmail.com | LinkedIn | GitHub| Medium | Website 
 
Objective 
An aspiring AI developer and software engineer with a passion for building intelligent systems 
that automate and enhance real-world processes. Experienced in developing AI-powered 
solutions, fine-tuning models, and deploying applications on cloud infrastructure. Skilled in 
integrating natural language processing (NLP), computer vision, and large language models into 
innovative projects. 
 
Education 
CGPA: 6.9 
 
University of Lucknow | Bachelor of Computer Application                       2022-2025 
 
Relevant Coursework 
Data Structures, Software Engineering, Algorithms, Database Management, Computer Networks, 
Operating System, Artificial Intelligence, Machine Learning 
Skills & abilities 
 
• 
Programming Languages: Python 
• 
AI/ML Frameworks: Hugging Face, Langhian, Llama Index, Unsloth, Transformers, FastAPI, 
Numpy, pandas, seaborn, P

In [ ]:
from langchain_core.output_parsers import XMLOutputParser, JsonOutputParser

In [8]:
from prompts import resume_parser_prompt
import re
from langchain_core.runnables import RunnableLambda

def clean_XML(text):
    text = re.sub(r'[^\x09\x0A\x0D\x20-\x7F]', '', text)
    return text

def Parse_resume(resume_data:str):

    logger.info('Node: Make Question started')
    data = resume_data
    parser = JsonOutputParser()
    parser.get_format_instructions()
    prompt = PromptTemplate(
        template=(resume_parser_prompt),
        input_variables=["RESUME"],
        partial_variables={"format_instructions": parser.get_format_instructions()},

    )

    chain = prompt | llm |parser
    parsed_resume = chain.invoke({"RESUME": data})
    return parsed_resume
res = Parse_resume(resume_data)
print(res)


2025-07-16 09:38:54,107 - __main__ - INFO - Node: Make Question started
2025-07-16 09:38:56,009 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


{'full_name': 'Shaswat Singh', 'email': 'Shaswatsingh910@gmail.com', 'phone': '9140529926', 'linkedin': '', 'github': '', 'portfolio_website': 'Website', 'location': 'Lucknow, UP, India', 'education': [{'degree': 'Bachelor of Computer Application', 'field_of_study': '', 'university': 'University of Lucknow', 'start_year': '2022', 'end_year': '2025'}], 'work_experience': [], 'skills': ['Python', 'Hugging Face', 'Langhian', 'Llama Index', 'Unsloth', 'Transformers', 'FastAPI', 'Numpy', 'pandas', 'seaborn', 'PyTorch', 'vllm', 'LM-Deploy', 'Git', 'Ollama', 'Ngrok', 'Jupyter Notebook', 'Vercel', 'ChromaDB', 'MySQL', 'Qdrant', 'Flask', 'Web-Dev'], 'certifications': [], 'projects': [{'name': 'DevEcho | AI-Powered Social Media Assistant', 'description': 'An intelligent assistant that automates LinkedIn post generation from user-provided content using LLMs and Retrieval-Augmented Generation (RAG). Integrated data collection pipelines for topical research.', 'technologies': ['Python', 'LangChain'

In [12]:
print(res)

{'full_name': 'Shaswat Singh', 'email': 'Shaswatsingh910@gmail.com', 'phone': '9140529926', 'linkedin': '', 'github': '', 'portfolio_website': 'Website', 'location': 'Lucknow, UP, India', 'education': [{'degree': 'Bachelor of Computer Application', 'field_of_study': '', 'university': 'University of Lucknow', 'start_year': '2022', 'end_year': '2025'}], 'work_experience': [], 'skills': ['Python', 'Hugging Face', 'Langhian', 'Llama Index', 'Unsloth', 'Transformers', 'FastAPI', 'Numpy', 'pandas', 'seaborn', 'PyTorch', 'vllm', 'LM-Deploy', 'Git', 'Ollama', 'Ngrok', 'Jupyter Notebook', 'Vercel', 'ChromaDB', 'MySQL', 'Qdrant', 'Flask', 'Web-Dev'], 'certifications': [], 'projects': [{'name': 'DevEcho | AI-Powered Social Media Assistant', 'description': 'An intelligent assistant that automates LinkedIn post generation from user-provided content using LLMs and Retrieval-Augmented Generation (RAG). Integrated data collection pipelines for topical research.', 'technologies': ['Python', 'LangChain'

In [13]:
import json
with open ("resume.json", "w") as f:
    json.dump(res, f)

## AGENT greet

In [3]:
from prompts import greet_prompt_template
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
        template=(greet_prompt_template),
        input_variables=["candidate_name", "agent_name", "hr_manager_name"],
        # partial_variables={"format_instructions": parser.get_format_instructions()},
    )


chain = prompt | llm

res = chain.invoke({"input":"START","candidate_name": "John Doe", "agent_name":"janus", "hr_manager_name": "radhika"})
print(res.content)

2025-07-16 11:22:10,479 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Hi, I'm janus, an AI assistant working under radhika. My role is to help evaluate and shortlist candidates in a structured, unbiased, and data-driven way.  

Please upload a resume or provide candidate details so I can begin the evaluation process.  




In [16]:
from prompts import greet_prompt_template
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
        template=(greet_prompt_template),
        input_variables=["input","candidate_name", "agent_name", "hr_manager_name"]
        # partial_variables={"format_instructions": parser.get_format_instructions()},
    )


chain = prompt | llm

res = chain.invoke({" input":"START","candidate_name":"Shaswat","agent_name":"HireBot","hr_manager_name":"Ms. Priya Sharma"})
print(res.content)

2025-07-16 11:11:49,586 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Understood! I am ready to assist you in evaluating candidates.  

Please provide me with the resume or candidate details. I will follow your instructions carefully and provide objective evaluations based on the information given.  




In [ ]:
def make_question():
    """Generate exactly 3 questions that help understand the user's research intent from the query.
    The questions should be easy to answer in one word or a short sentence, free of jargon and complexity,
    and focused on enriching the research plan (goal, focus area, scope, time)."""

    logger.info('Node: Make Question started')
    query = state["initial_query"]
    questions = {}
    num = 3
    parser = JsonOutputParser()
    prompt = PromptTemplate(
        template=(Questionnaire_prompt),
        input_variables=["num", "query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | llm | parser

    try:
        logger.info(f'Node: Making questions on {query}')
        questions = chain.invoke({"num": num, "query": query})
        logger.info(f'Generated questions: {questions}')    
        if not isinstance(questions, dict):
            logger.info(f'Node: LLM generated questions are not a dictionary: Output: {questions}')
            return{"query_questions": {}}    
        return {"query_questions": questions}
    except Exception as e:
        logger.error(f"Failed to generate questions: {e}")
        return {"query_questions": {}}


## Generate tech questions

In [17]:

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from prompts import tech_questions



def Tech_question(resume_data):
    """Generates clarifying questions based on the initial query using an LLM."""
    # logger.info("Node: make_question execution started.")
    questions = {}
    num = 3 # Number of questions to generate
    parser = JsonOutputParser()
    # Corrected Example Output to be valid JSON
    prompt = PromptTemplate(
        template=(tech_questions),
        input_variables=["num", "resume_data"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | llm | parser


    questions = chain.invoke({"num": num, "tech_stack": resume_data})
    return questions

result = Tech_question(resume_data)
type(result)

dict

In [22]:
print(result)

{'q1': 'Describe the role of LangChain in building an AI-powered application.', 'q2': 'How do you utilize ChromaDB for efficient knowledge retrieval in your AI projects?', 'q3': 'Explain the difference between using PyTorch and Transformers for building a machine learning model.'}


In [19]:
ques = result

## Get answers 

In [29]:
def answers(questions):
    answers = {}
    for i in questions:
        answers[questions[i]] = input(f"Question {i}: {questions[i]}")
    print(answers)
    print(type(answers))
    return answers
        

In [31]:
start = {}
start = answers(ques)


{'Describe the role of LangChain in building an AI-powered application.': '45', 'How do you utilize ChromaDB for efficient knowledge retrieval in your AI projects?': '45', 'Explain the difference between using PyTorch and Transformers for building a machine learning model.': '4'}
<class 'dict'>


In [2]:
sim_answers = {
    "Describe the role of LangChain in building an AI-powered application.": 
        "LangChain acts as an orchestration framework for building AI-powered applications by enabling developers to connect language models with external data sources and tools. It provides abstractions for prompt templates, chains, agents, memory, and tools, making it easier to manage complex workflows involving LLMs. In essence, it simplifies integrating and controlling how LLMs interact with documents, APIs, databases, or user input in a structured way.",

    "How do you utilize ChromaDB for efficient knowledge retrieval in your AI projects?": 
        "ChromaDB is used as a vector database for storing and retrieving embeddings of documents or knowledge chunks. In AI projects, it enables efficient semantic search by allowing fast similarity searches using vector representations of queries. This makes it ideal for Retrieval-Augmented Generation (RAG) pipelines, where relevant documents are retrieved from ChromaDB based on the user’s query and then passed to the language model for context-aware response generation.",

    "Explain the difference between using PyTorch and Transformers for building a machine learning model.": 
        "PyTorch is a deep learning framework used to build and train custom neural network models from scratch. It provides low-level control over model architecture, training loops, and optimization. On the other hand, the Transformers library (by Hugging Face) is a high-level library built on top of PyTorch (and optionally TensorFlow) that provides pre-trained transformer-based models like BERT, GPT, etc., with simple APIs for fine-tuning and inference. In summary, PyTorch is foundational and flexible, while Transformers is specialized and optimized for NLP tasks using state-of-the-art models."
}


In [3]:
print(sim_answers)

{'Describe the role of LangChain in building an AI-powered application.': 'LangChain acts as an orchestration framework for building AI-powered applications by enabling developers to connect language models with external data sources and tools. It provides abstractions for prompt templates, chains, agents, memory, and tools, making it easier to manage complex workflows involving LLMs. In essence, it simplifies integrating and controlling how LLMs interact with documents, APIs, databases, or user input in a structured way.', 'How do you utilize ChromaDB for efficient knowledge retrieval in your AI projects?': 'ChromaDB is used as a vector database for storing and retrieving embeddings of documents or knowledge chunks. In AI projects, it enables efficient semantic search by allowing fast similarity searches using vector representations of queries. This makes it ideal for Retrieval-Augmented Generation (RAG) pipelines, where relevant documents are retrieved from ChromaDB based on the user

## Get results 

In [6]:
from prompts import Evaluator_prompt
def Get_reults(answers: dict):
    """Generates clarifying questions based on the initial query using an LLM."""
 

    # parser = JsonOutputParser()
    
    prompt = PromptTemplate(
        template=(Evaluator_prompt),
        input_variables=["qa"],
        # partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | llm 


    questions = chain.invoke({"qa": answers})
    return questions

result = Get_reults(sim_answers)


2025-07-18 13:50:11,053 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [7]:
print(type(result))

<class 'langchain_core.messages.ai.AIMessage'>


In [8]:
print(result.content)

Q1: TC=5, AC=5, DC=5
Q2: TC=5, AC=5, DC=5
Q3: TC=5, AC=5, DC=5
Total Score: 45 out of 45


## Sendoff

In [8]:
from prompts import sendoff_node_prompt
from langchain_core.prompts import PromptTemplate

def sendoff_node(candidate_name):

    message = f"""
    Thank you for completing the initial steps of the hiring process, {candidate_name}. 🙌
    Our team will carefully evaluate your responses and resume.
    You will be notified via email or message if you are selected for the next round.
    We appreciate your patience and interest in this opportunity!
    """

    print(message)
    
    
sendoff_node(candidate_name = "Jan")


    Thank you for completing the initial steps of the hiring process, Jan. 🙌
    Our team will carefully evaluate your responses and resume.
    You will be notified via email or message if you are selected for the next round.
    We appreciate your patience and interest in this opportunity!
    
